In [1]:
# 데이터 전처리
# 1. 엑셀 파일 읽어오기 (pandas, Ipython.display.display)
# 2. 문장(x), 감정(t)로 나누기
# 3. 잘못된 감정 데이터를 가진 열 삭제( 정상 : ['공포', '놀람', '분노', '슬픔, '중립', '행복', '혐오'] )
# 4. 문장(x)을 형태소 단위로 끊기 (tokenize) (+ 특문 처리) (+ 불용어 처리) (kiwipiepy.Kiwi(), custom.text_preprocess_kor)
# 5. 각 문장의 형태소 갯수 세기 (padding 기준 정히기 위해서)
# 6. 데이터 저장

In [2]:
# 필요 모듈 임포트
import os
import sys
sys.path.append(os.path.join(os.path.dirname(""), ".."))
import custom

from tqdm import tqdm
import pandas
from IPython.display import display
from kiwipiepy import Kiwi

kiwi = Kiwi() #클래스 생성

In [3]:
def preprocess(ori_df, texts_col, moods_col) : # 기존 데이터프레임, 대상이 되는 컬럼 이름 파라미터로 처리
    # 필요한 열만 골라내기
    df = ori_df[[texts_col, moods_col]]
    
    # 컬럼 이름도 바꾸기
    df.columns = ["text", "mood"]
    
    # display(df)
    # print(df["mood"].unique())
    
    # 잘못된 감정값 가진 열 제거
    wrong_list = []
    mood_list = ['공포', '놀람', '분노', '슬픔', '중립', '행복', '혐오']
    
    for a in df["mood"].unique().tolist() :
        if a not in mood_list :
            wrong_list.append(a)
    
    for a in wrong_list :
        df = df[df['mood'] != a]

    df = df[df["mood"].isnull() == False ] #감정 null값 제거
    
    # display(df)
    
    # 문장을 형태소 단위로 끊기
    
    for i in tqdm(range(len(df))) :
        text = str(df.iloc[i,0]) # 텍스트 하나를 df에서 가져와서
        text = custom.text_preprocess_kor(text, True, False) # 문장 끝 . ? ! 는 남기고, 초성은 제거
        word_list = [] # 텍스트의 형태소를 저장할 임시 리스트 변수 선언
        temp = kiwi.tokenize(text, z_coda=True, split_complex=True, compatible_jamo=True) # 형태소 단위로 문장 분리
        #z_coda : 어미 끝 밭침 분리, split_complex : 합성어 분리, compatible_jamo : 밭침 분리 할 때, 자음 통합
        for w in temp :
            word_list.append(w.form) # 형태소를 임시 리스트에 추가
        text = " ".join(word_list) # 형태소 리스트를 다시 한 문장으로 합치고
        df.iloc[i,0] = text
    
    # display(df)
    
    # 형태소의 갯수 세서 df에 저장
    
    word_len = [] #형태소 갯수 저장하는 임시 리스트
    for i in tqdm(range(len(df))) :
        text = df.iloc[i,0]
        word_len.append(len(text.split()))
    df['word_len'] = word_len
    
    return df

In [4]:
# 엑셀 파일 읽어오기
file_name = "mood_data/mood1.xlsx"

ori_df = pandas.read_excel(file_name, header = 0)

ori_df.info()

display(ori_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38594 entries, 0 to 38593
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Sentence    38594 non-null  object 
 1   Emotion     38594 non-null  object 
 2   Unnamed: 2  0 non-null      float64
 3   Unnamed: 3  0 non-null      float64
 4   Unnamed: 4  0 non-null      float64
 5   공포          7 non-null      object 
 6   5468        7 non-null      float64
dtypes: float64(4), object(3)
memory usage: 2.1+ MB


,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
0,언니 동생으로 부르는게 맞는 일인가요..??,공포,NaN,NaN,NaN,놀람,5898.0
1,그냥 내 느낌일뿐겠지?,공포,NaN,NaN,NaN,분노,5665.0
2,아직너무초기라서 그런거죠?,공포,NaN,NaN,NaN,슬픔,5267.0
3,유치원버스 사고 낫다던데,공포,NaN,NaN,NaN,중립,4830.0
4,근데 원래이런거맞나요,공포,NaN,NaN,NaN,행복,6037.0
...,...,...,...,...,...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오,NaN,NaN,NaN,NaN,NaN
38590,재미가 없으니 망하지,혐오,NaN,NaN,NaN,NaN,NaN
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오,NaN,NaN,NaN,NaN,NaN
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오,NaN,NaN,NaN,NaN,NaN


In [5]:
df = preprocess(ori_df, "Sentence", "Emotion")

display(df)

df.to_csv("mood1.csv", encoding="UTF8", sep = "\t", index=False)

100%|█████████████████████████████████████████████████████████████████████████| 38594/38594 [00:00<00:00, 60407.40it/s]


,text,mood,word_len
0,언니 동생 으로 부르 는 것 이 맞 는 일 이 ㄴ가 요 ?,공포,14
1,그냥 나 의 느낌 이 ㄹ 뿐 겠 지 ?,공포,10
2,아직 너무 초기 이 라서 그런 거 이 죠 ?,공포,10
3,유치원 버스 사고 나 엇 다던데,공포,6
4,근데 원래 이런 거 맞 나 요,공포,7
...,...,...,...
38589,솔직 히 예보 제대로 못 하 는 데 세금 이 라도 아끼 게 그냥 폐지 하 어라 .,혐오,18
38590,재미 가 없 으니 망하 지,혐오,6
38591,공장 도시락 비우생 적 이 ㅁ 아르바이트 하 었 는데 화장실 가 어서 ㅇ 손 도 않...,혐오,32
38592,코딱지 만 하 ㄴ 나라 에서 지 들 끼리 피 터지 게 싸우 는 세 ㄴ지 ㅇ 클래스,혐오,18


In [6]:
# 엑셀 파일 읽어오기
file_name = "mood_data/mood2.xlsx"

ori_df = pandas.read_excel(file_name, header = 1)

ori_df.info()

display(ori_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55628 entries, 0 to 55627
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   dialog #     10000 non-null  object 
 1   발화           55627 non-null  object 
 2   감정           55621 non-null  object 
 3   Unnamed: 3   0 non-null      float64
 4   Unnamed: 4   0 non-null      float64
 5   1030         0 non-null      float64
 6   Unnamed: 6   0 non-null      float64
 7   Unnamed: 7   0 non-null      float64
 8   Unnamed: 8   0 non-null      float64
 9   Unnamed: 9   0 non-null      float64
 10  Unnamed: 10  0 non-null      float64
 11  Unnamed: 11  0 non-null      float64
dtypes: float64(9), object(3)
memory usage: 5.1+ MB


,dialog #,발화,감정,Unnamed: 3,Unnamed: 4,1030,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,S,아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!,분노,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.,혐오,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,손님 왔어요.,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,손님? 누구?,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,몰라요. 팀장님 친구래요.,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
55623,S,얘긴 다 끝났냐? 원예부,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55624,NaN,"예. 그거 때문에, 부탁이 있......는......데요.",중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55625,NaN,여자 숨겨달라는거면 사절이다.,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55626,NaN,아무래도 안되나요?,중립,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df = preprocess(ori_df, "발화", "감정")

display(df)

df.to_csv("mood2.csv", encoding="UTF8", sep = "\t", index=False)

100%|█████████████████████████████████████████████████████████████████████████| 55600/55600 [00:01<00:00, 50211.25it/s]


,text,mood,word_len
0,아 진짜 ! 사무실 에서 피 지 말 라니까 ㄴ ! 간접흡연 이 얼마나 안 좋 은데 !,분노,18
1,그럼 직접 흡연 하 는 나 ㄴ 얼마나 안 좋 겠 니 ? 안 그렇 어 ? 보 면 꼭 ...,혐오,27
2,손 님 오 었 어 요 .,중립,7
3,손 님 ? 누구 ?,중립,5
4,모르 어 요 . 팀장 님 친구 이 래 요 .,중립,11
...,...,...,...
55623,얘기 ㄴ 다 끝나 었 냐 ? 원예 부,중립,9
55624,예 . 그거 때문 에 부탁 이 있 . 는 . 데 요 .,중립,14
55625,여자 숨기 어 달 라는 거 이 면 사절 이 다 .,중립,12
55626,아무래도 안 되 나 요 ?,중립,6


In [9]:
# 데이터 다시 불러와서 합치기

df1 = pandas.read_csv("mood1.csv", encoding="UTF8", sep = "\t")
df2 = pandas.read_csv("mood2.csv", encoding="UTF8", sep = "\t")

new_df = pandas.concat([df1, df2])

display(new_df)

,text,mood,word_len
0,언니 동생 으로 부르 는 것 이 맞 는 일 이 ㄴ가 요 ?,공포,14
1,그냥 나 의 느낌 이 ㄹ 뿐 겠 지 ?,공포,10
2,아직 너무 초기 이 라서 그런 거 이 죠 ?,공포,10
3,유치원 버스 사고 나 엇 다던데,공포,6
4,근데 원래 이런 거 맞 나 요,공포,7
...,...,...,...
55595,얘기 ㄴ 다 끝나 었 냐 ? 원예 부,중립,9
55596,예 . 그거 때문 에 부탁 이 있 . 는 . 데 요 .,중립,14
55597,여자 숨기 어 달 라는 거 이 면 사절 이 다 .,중립,12
55598,아무래도 안 되 나 요 ?,중립,6


In [10]:
# 감정 원 핫 인코딩

new_df = pandas.get_dummies(new_df, columns=['mood'])

display(new_df)

,text,word_len,mood_공포,mood_놀람,mood_분노,mood_슬픔,mood_중립,mood_행복,mood_혐오
0,언니 동생 으로 부르 는 것 이 맞 는 일 이 ㄴ가 요 ?,14,True,False,False,False,False,False,False
1,그냥 나 의 느낌 이 ㄹ 뿐 겠 지 ?,10,True,False,False,False,False,False,False
2,아직 너무 초기 이 라서 그런 거 이 죠 ?,10,True,False,False,False,False,False,False
3,유치원 버스 사고 나 엇 다던데,6,True,False,False,False,False,False,False
4,근데 원래 이런 거 맞 나 요,7,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
55595,얘기 ㄴ 다 끝나 었 냐 ? 원예 부,9,False,False,False,False,True,False,False
55596,예 . 그거 때문 에 부탁 이 있 . 는 . 데 요 .,14,False,False,False,False,True,False,False
55597,여자 숨기 어 달 라는 거 이 면 사절 이 다 .,12,False,False,False,False,True,False,False
55598,아무래도 안 되 나 요 ?,6,False,False,False,False,True,False,False


In [11]:
# 합친 데이터 저장

new_df.to_csv("mood.csv", encoding="UTF8", sep="\t", index=False)